In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

26/01/27 02:17:46 WARN Utils: Your hostname, codespaces-f3c854 resolves to a loopback address: 127.0.0.1; using 10.0.2.53 instead (on interface eth0)
26/01/27 02:17:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/27 02:17:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# q1 Spark Version

In [9]:
spark

In [10]:
spark.version

'3.3.2'

# q2 Repartition into 4 partitions, AND save to parquet

In [2]:
!pwd

/workspaces/data-engineering-zoomcamp-homework/05-batch/code


In [ ]:
# 2026 w5 homework parquet file path
# /workspaces/data-engineering-zoomcamp-homework/05-batch/code/yellow_tripdata_2024-10.parquet

In [5]:
df_yellow = spark.read \
                 .option("header", "true") \
                 .option("samplingRatio", 0.01) \
                 .option("inferSchema", True) \
                 .parquet('yellow_tripdata_2024-10.parquet')

In [6]:
df_yellow.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df_yellow.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [16]:
year = 2024
month = 10
output_path = f'data/pq/yellow/{year}/{month:02d}/'

output_path

'data/pq/yellow/2024/10/'

In [17]:
df_yellow \
        .repartition(4) \
        .write.parquet(output_path, mode="overwrite")

In [14]:
!ls -lh

total 904M
-rw-rw-rw-  1 codespace codespace  12K Jan 25 10:28 03_test.ipynb
-rw-rw-rw-  1 codespace codespace  80K Jan 26 03:25 04_pyspark.ipynb
-rw-rw-rw-  1 codespace codespace  48K Jan 26 03:46 05_taxi_schema.ipynb
-rw-rw-rw-  1 codespace codespace  30K Jan 26 03:56 06_spark_sql.ipynb
-rw-rw-rw-  1 codespace codespace 2.5K Jan 24 08:56 06_spark_sql.py
-rw-rw-rw-  1 codespace codespace 2.6K Jan 24 08:56 06_spark_sql_big_query.py
-rw-rw-rw-  1 codespace codespace  21K Jan 26 08:46 07_groupby_join.ipynb
-rw-rw-rw-  1 codespace codespace 139K Jan 26 09:45 08_rdds.ipynb
-rw-rw-rw-  1 codespace codespace 4.3K Jan 24 08:56 09_spark_gcs.ipynb
-rw-rw-rw-  1 codespace codespace 4.6K Jan 26 14:44 cloud.md
drwxrwxrwx+ 5 codespace codespace 4.0K Jan 26 03:53 data
-rwxrwxrwx  1 codespace codespace  615 Jan 26 02:43 download_data.sh
drwxr-xr-x+ 3 codespace codespace 4.0K Jan 25 10:32 fhvhv
-rw-rw-rw-  1 codespace codespace 718M Jan 25 10:30 fhvhv_tripdata_2021-01.csv
-rw-rw-rw-  1 codespace codes

In [18]:
!ls -lh data/pq/yellow/2024/10/

total 100M
-rw-r--r-- 1 codespace codespace   0 Jan 27 02:43 _SUCCESS
-rw-r--r-- 1 codespace codespace 25M Jan 27 02:43 part-00000-468d7f16-8a9f-4ca9-bff6-3ed89f438424-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 25M Jan 27 02:43 part-00001-468d7f16-8a9f-4ca9-bff6-3ed89f438424-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 25M Jan 27 02:43 part-00002-468d7f16-8a9f-4ca9-bff6-3ed89f438424-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 25M Jan 27 02:43 part-00003-468d7f16-8a9f-4ca9-bff6-3ed89f438424-c000.snappy.parquet


In [19]:
# now read from the parquet with 4 partitions
df_yellow = spark.read \
                 .option("header", "true") \
                 .option("samplingRatio", 0.01) \
                 .option("inferSchema", True) \
                 .parquet('data/pq/yellow/2024/10/*')

# q3 Count records 15 Oct

In [23]:
from pyspark.sql.functions import col, to_date, lit

In [20]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [25]:
df_yellow_w_renamed_dt = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')
    

In [26]:
df_yellow_w_renamed_dt.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [30]:
# 2024-10-15
df_yellow_w_renamed_dt.filter(to_date(df_yellow_w_renamed_dt.pickup_datetime) == lit('2024-10-15')).show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2|2024-10-15 00:43:57|2024-10-15 00:45:14|              1|         0.29|         5|                 N|         138|         223|           1|       65.0|  5.0|    0.0|       0.0|         0

In [32]:
df_yellow_w_renamed_dt.filter(to_date(df_yellow_w_renamed_dt.pickup_datetime) == '2024-10-15').show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2|2024-10-15 00:43:57|2024-10-15 00:45:14|              1|         0.29|         5|                 N|         138|         223|           1|       65.0|  5.0|    0.0|       0.0|         0

In [31]:
df_yellow_w_date = df_yellow_w_renamed_dt \
                   .withColumn('pickup_date', to_date(df_yellow_w_renamed_dt.pickup_datetime)) \
                   .withColumn('dropoff_date', to_date(df_yellow_w_renamed_dt.dropoff_datetime))
df_yellow_w_date.show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+
|       2|2024-10-12 04:16:59|2024-10-12 04:24:37|              2|         2.07|         1|                 N|         12

In [35]:
df_yellow_20241015 = df_yellow_w_date.filter(df_yellow_w_date.pickup_date == '2024-10-15')
df_yellow_20241015.show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+
|       2|2024-10-15 00:43:57|2024-10-15 00:45:14|              1|         0.29|         5|                 N|         13

In [36]:
df_yellow_20241015.count()

128893

# Question 4: Longest trip

In [38]:
# find diff of dropoff_datetime - pickup_datetime
df_yellow_w_date.withColumn('duration', df_yellow_w_date.dropoff_datetime - df_yellow_w_date.pickup_datetime).show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|            duration|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------------------+
|       2|2024-10-12 04:16:59|2024-10-12 04:24:37|        

In [42]:
# find diff of dropoff_datetime - pickup_datetime
# convert to hours (divide by 60 seconds, 60 minutes)
df_yellow_w_date.withColumn('duration_hours', (df_yellow_w_date.dropoff_datetime - df_yellow_w_date.pickup_datetime).cast('int') / 60/60).show(10)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|      duration_hours|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------------------+
|       2|2024-10-12 04:16:59|2024-10-12 04:24:37|        

In [43]:
# find max duration_hours
df_yellow_w_date.withColumn('duration_hours', (df_yellow_w_date.dropoff_datetime - df_yellow_w_date.pickup_datetime).cast('int') / 60/60).agg({'duration_hours': 'max'}).show()

[Stage 32:>                                                         (0 + 2) / 2]

+-------------------+
|max(duration_hours)|
+-------------------+
|  162.6177777777778|
+-------------------+



# Question 6: Least frequent pickup location zone


In [46]:
df_yellow_w_date.groupBy('PULocationID').count().orderBy('count', ascending=False).show()

+------------+------+
|PULocationID| count|
+------------+------+
|         132|194113|
|         237|191011|
|         161|177568|
|         236|167231|
|         162|132055|
|         230|127529|
|         142|124801|
|         186|121396|
|         138|118708|
|         170|110194|
|         163|108517|
|         234|104874|
|          68|103753|
|         239|102386|
|          48| 91864|
|         164| 86562|
|         249| 86242|
|         141| 84690|
|          79| 84429|
|         107| 75192|
+------------+------+
only showing top 20 rows



In [47]:
df_yellow_pickuplocationid_count = df_yellow_w_date.groupBy('PULocationID').count().orderBy('count', ascending=False)

In [51]:
df_yellow_pickuplocationid_count.schema

StructType([StructField('PULocationID', IntegerType(), True), StructField('count', LongType(), False)])

In [49]:
df_zonelookup = spark.read \
                 .option("header", "true") \
                 .option("samplingRatio", 0.01) \
                 .option("inferSchema", True) \
                 .csv('taxi_zone_lookup.csv')
df_zonelookup.show(10)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 10 rows



In [50]:
df_zonelookup.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [52]:
df_yellow_pickuplocationwithname_count = df_yellow_pickuplocationid_count.join(df_zonelookup,   # df1.join(df2
                                         df_yellow_pickuplocationid_count.PULocationID == df_zonelookup.LocationID,   # joining key
                                         'left')
df_yellow_pickuplocationwithname_count.show(10)

+------------+------+----------+---------+--------------------+------------+
|PULocationID| count|LocationID|  Borough|                Zone|service_zone|
+------------+------+----------+---------+--------------------+------------+
|         132|194113|       132|   Queens|         JFK Airport|    Airports|
|         237|191011|       237|Manhattan|Upper East Side S...| Yellow Zone|
|         161|177568|       161|Manhattan|      Midtown Center| Yellow Zone|
|         236|167231|       236|Manhattan|Upper East Side N...| Yellow Zone|
|         162|132055|       162|Manhattan|        Midtown East| Yellow Zone|
|         230|127529|       230|Manhattan|Times Sq/Theatre ...| Yellow Zone|
|         142|124801|       142|Manhattan| Lincoln Square East| Yellow Zone|
|         186|121396|       186|Manhattan|Penn Station/Madi...| Yellow Zone|
|         138|118708|       138|   Queens|   LaGuardia Airport|    Airports|
|         170|110194|       170|Manhattan|         Murray Hill| Yellow Zone|

In [53]:
# NOW sort in ascending order for count
df_yellow_pickuplocationwithname_count.orderBy('count', ascending=True).show(10)

+------------+-----+----------+-------------+--------------------+------------+
|PULocationID|count|LocationID|      Borough|                Zone|service_zone|
+------------+-----+----------+-------------+--------------------+------------+
|         105|    1|       105|    Manhattan|Governor's Island...| Yellow Zone|
|         199|    2|       199|        Bronx|       Rikers Island|   Boro Zone|
|           5|    2|         5|Staten Island|       Arden Heights|   Boro Zone|
|         111|    3|       111|     Brooklyn| Green-Wood Cemetery|   Boro Zone|
|           2|    3|         2|       Queens|         Jamaica Bay|   Boro Zone|
|          44|    4|        44|Staten Island|Charleston/Totten...|   Boro Zone|
|          84|    4|        84|Staten Island|Eltingville/Annad...|   Boro Zone|
|         204|    4|       204|Staten Island|   Rossville/Woodrow|   Boro Zone|
|         245|    4|       245|Staten Island|       West Brighton|   Boro Zone|
|         187|    4|       187|Staten Is